# Origen de datos

El origen de los datos es un proyecto de Kaggle. Una descripción se puede encontrar aquí:

https://www.kaggle.com/dcohen21/8anu-climbing-logbook

En este caso **se trata de una base de datos sqlite3** comprimida en zip. Contiene la información obtenida mediante webscraping del sitio www.8a.nu

Este sítio basicamente es un portal de escaladores en el que los usuarios pueden logarse e ir subiendo sus ascensos para compartir con otros su progresión y experiencias sobres las vías que escala.

En este notebook voy a realizar dos tareas principalmente:

- Explorar la base de datos para ver que información (tablas) proporciona.
- Generar ficheros CSV y un excel "limpios" con la información necesaria para el modelo planteado.
- Plantear las preguntas que voy a querer resolver en la práctica.


# Modelo conceptual

Las entidades del modelo buscado son:

![](./data/ModeloConceptual.png)


# Modelo de relación

Las entidades se relacionan de la siguiente manera

![](./data/ModeloRelacional.png)


# Obtencion y limpieza de la información

Para explorar la información que contiene esa base de datos obtenida por webscraping sigo los siguientes pasos:

## 1. Descomprimimos la base de datos obtenida de Kaggle

In [3]:
import io
import os
from zipfile import ZipFile

dataDirectory = './data/'
filePath = dataDirectory + "8anu-climbing-logbook.zip"

if not os.path.exists(filePath):
    os.makedirs(dataDirectory)
    print ("Descomprimiendo base de datos...")
    
    with zipfile.ZipFile(filePath, "r") as zipStream:
        zipStream.extractall(dataDirectory)
    
    print ("Hecho!")

## 2. Exploración la base de datos sqlite: identificación de tablas

Para ello conectamos con la base de datos y lanzamos la query que me dice qué tablas hay definidas en la base de datos.

In [7]:

import sqlite3
import pandas as pd


dbConnection = sqlite3.connect(dataDirectory + 'database.sqlite')

pd.read_sql_query('''
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
''', dbConnection)




,name
0,ascent
1,grade
2,method
3,user


La base de datos tiene cuatro tablas que, por el nombre y conociendo un poco el mundillo de la escalada, parece que van a contener la siguiente información:

- **ascent**: Tabla de ascensos
- **grade**: Tabla de grados de dificultad de los ascensos
- **method**: Tabla de metodos de ascenso
- **user**: Tabla de usuarios (escaladores)

En los siguientes apartados vamos a ir tabla a tabla limpiando y obteniendo un CSV lo más limpio posible.

## 3. Construccion de los datos de escaladores

De la tabla de usuarios obtendré los datos de la tabla de escaladores.

El numero de usuarios registrados es el siguiente:

In [32]:
pd.read_sql_query(
    '''
    SELECT count()
    FROM user;
    ''', 
    dbConnection)


,count()
0,62593


Si visualizamos los campos de la tabla vemos que existen:

In [9]:
df = pd.read_sql_query(
    '''
    SELECT *
    FROM user
    LIMIT 1
    ''',
    dbConnection);

list(df)

['id',
 'first_name',
 'last_name',
 'city',
 'country',
 'sex',
 'height',
 'weight',
 'started',
 'competitions',
 'occupation',
 'sponsor1',
 'sponsor2',
 'sponsor3',
 'best_area',
 'worst_area',
 'guide_area',
 'interests',
 'birth',
 'presentation',
 'deactivated',
 'anonymous']

In [12]:
df.head(20)

,id,first_name,last_name,city,country,sex,height,weight,started,competitions,...,sponsor2,sponsor3,best_area,worst_area,guide_area,interests,birth,presentation,deactivated,anonymous
0,1,Leif,Jägerbrand,Göteborg,SWE,0,177,73,1996,,...,,,"Railay beach, Krabi, Thailand",None,,,1976-03-10,,0,0


De esta información me quedaré sólo con las columnas útiles para mi modelo:
1. id
2. first_name
3. last_name
4. city
5. country
6. sex
7. started
8. birth
9. occupation

Para ello hago una query tomando solo esos campos sobre toda la tabla y lo llevo a un CSV

In [14]:
df = pd.read_sql_query(
    '''
    SELECT id,
           first_name,
           last_name,
           CASE WHEN sex=0 THEN 'Hombre'
                WHEN sex=1 THEN 'Mujer'
                ELSE NULL
           END as "sexo",
           birth as "fecha_nacimiento",
           city as "ciudad",
           country as "pais",
           started as "año_comienzo"
    FROM user
    ''',
    dbConnection);

df["nombre"] = df["first_name"] + " " + df["last_name"];
df.drop(["first_name", "last_name"], axis=1, inplace=True);

df.to_csv(dataDirectory+'users.csv', index=False);

df.head()

,id,sexo,fecha_nacimiento,ciudad,pais,año_comienzo,nombre
0,1,Hombre,1976-03-10,Göteborg,SWE,1996,Leif Jägerbrand
1,2,Hombre,None,stockholm,SWE,2000,Andreas Collisch
2,3,Hombre,1973-09-09,Umeå,SWE,1995,Magnus Öberg
3,4,Mujer,1984-07-26,Goteborg,SWE,2001,Annika Frodi-Lundgren (f)
4,5,Hombre,1969-05-07,North Attleboro,USA,1991,Joe McLoughlin


### Tabla de grados

In [86]:
pd.read_sql_query(
    '''
    SELECT *
    FROM grade
    LIMIT 10
    ''',
    dbConnection)

,id,score,fra_routes,fra_routes_input,fra_routes_selector,fra_boulders,fra_boulders_input,fra_boulders_selector,usa_routes,usa_routes_input,usa_routes_selector,usa_boulders,usa_boulders_input,usa_boulders_selector
0,1,0,-,1,1,-,1,1,3/4,1,1,VB,1,1
1,2,0,1,0,0,1,0,0,,0,0,,0,0
2,3,0,1a,0,0,1A,0,0,,0,0,,0,0
3,4,0,1b,0,0,1B,0,0,,0,0,,0,0
4,5,0,1c,0,0,1C,0,0,,0,0,,0,0
5,6,0,1+,0,0,1+,0,0,,0,0,,0,0
6,7,80,2,1,0,2,1,0,5.1,1,0,VB,0,0
7,8,0,2a,0,0,2A,0,0,,0,0,,0,0
8,9,0,2b,0,0,2B,0,0,,0,0,,0,0
9,10,0,2c,0,0,2C,0,0,,0,0,,0,0


In [84]:
df = pd.read_sql_query(
    '''
    SELECT id as "id_grade",
           fra_routes as "french_grade",
           usa_routes as "us_grade",
           usa_boulders as "us_boulder_grade"
    FROM grade
    ''',
    dbConnection);

df.to_csv(dataDirectory+'grades.csv', index=False);

### Tabla de métodos

In [69]:
pd.read_sql_query(
    '''
    SELECT *
    FROM method
    ''',
    dbConnection)

,id,score,shorthand,name
0,1,0,redpoint,Redpoint
1,2,53,flash,Flash
2,3,145,onsight,Onsight
3,4,-52,toprope,Toprope
4,5,95,onsight,Onsight


In [85]:
df = pd.read_sql_query(
    '''
    SELECT id as "id_method",
            name
    FROM method
    ''',
    dbConnection);

df.to_csv(dataDirectory+'methods.csv', index=False);

### Tabla de ascensos

Longitud de las tabla de acensos ('ascent'):

In [65]:
pd.read_sql_query(
    '''
    SELECT count()
    FROM ascent;
    ''', 
    dbConnection)

,count()
0,4111877


Campos de la tabla

In [97]:
df = pd.read_sql_query(
    '''
    SELECT *
    FROM ascent
    LIMIT 5
    ''',
    dbConnection);
list(df)

['id',
 'user_id',
 'grade_id',
 'notes',
 'raw_notes',
 'method_id',
 'climb_type',
 'total_score',
 'date',
 'year',
 'last_year',
 'rec_date',
 'project_ascent_date',
 'name',
 'crag_id',
 'crag',
 'sector_id',
 'sector',
 'country',
 'comment',
 'rating',
 'description',
 'yellow_id',
 'climb_try',
 'repeat',
 'exclude_from_ranking',
 'user_recommended',
 'chipped']

In [11]:
pd.read_sql_query(
    '''
    SELECT count()
    FROM ascent
    WHERE country IS NOT NULL AND country!=""
          AND crag IS NOT NULL AND crag!=""
          AND sector IS NOT NULL AND sector!=""
    ''',
    dbConnection)

,count()
0,2848596


In [12]:
df = pd.read_sql_query(
    '''
    SELECT id as "id_ascent",
           user_id as "id_user",
           grade_id as "id_grade",
           method_id as "id_method",
           name,
           crag,
           sector,
           date,
           country
    FROM ascent
    WHERE country IS NOT NULL AND country!=""
          AND crag IS NOT NULL AND crag!=""
          AND sector IS NOT NULL AND sector!=""
    ''',
    dbConnection);

df.to_csv(dataDirectory+'ascents.csv', index=False);

In [104]:
pd.to_datetime(917478000, unit='s')


Timestamp('1999-01-27 23:00:00')